In [40]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from topogram_client import TopogramAPIClient

from csv import DictReader
import os
import json
import pymongo
#import musicbrainzngs
import arrow
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict
from dateutil import parser
from datetime import timedelta
import pickle
import time
from slugify import slugify
from fuzzywuzzy import fuzz
import urllib
import math

##SET PATH TO SAVE FILES:
savpath="./bands_dicts_GZ/"


In [41]:
#a function to have a range of colors for nodes/edges 
import colorsys
def get_N_HexCol(N=20):

    HSV_tuples = [(x*1.0/N, 0.5, 0.5) for x in xrange(N)]
    hex_out = []
    for rgb in HSV_tuples:
        rgb = map(lambda x: int(x*255),colorsys.hsv_to_rgb(*rgb))
        hex_out.append("".join(map(lambda x: chr(x).encode('hex'),rgb)))
    return hex_out

In [42]:
def create_topogram(title, nodes, edges):

    print "Creating topogram '%s'"%title

    try :
        r = topogram.create_topogram(title)
    except ValueError:
        print '> Topogram already exists'
        r = topogram.get_topogram_by_name(title)
    print r
    topogram_ID = r["data"]["_id"]
    print "topogram ID : %s"%topogram_ID

    # get and backup existing nodes and edges
    existing_nodes = topogram.get_nodes(topogram_ID)["data"]
    existing_edges = topogram.get_edges(topogram_ID)["data"]

    # clear existing graph
    if len(existing_nodes):
        topogram.delete_nodes([n["_id"] for n in existing_nodes])
        print "%s nodes deleted"%len(existing_nodes)
    if len(existing_edges):
        topogram.delete_edges([n["_id"] for n in existing_edges])
        print "%s edges deleted"%len(existing_edges)

    r = topogram.create_nodes(topogram_ID, nodes)
    #print r
    print "%s nodes created."%len(r["data"])
    r = topogram.create_edges(topogram_ID, edges)
    print "%s edges created."%len(r["data"])

    print "done. Topogram has been updated. Check it at %s/topograms/%s"%(TOPOGRAM_URL, topogram_ID)



In [7]:
artist={}
artist["name"]="steve-aoki"
artist["mbid"]="a39acc4a-985e-4173-a9ed-f325f2d3bc1c"

with open("../../4_miner/tour_miner/"+slugify(artist["name"]+artist["mbid"])+".obj", 'rb') as handle:
          tempL= pickle.load( handle)
print tempL

{1: {'tourLength': 9, 'tourdist': 8537.241216520675, 'tourYear': 2008, 'tourdistOpt': 9340.712656690901}, 2: {'tourLength': 3, 'tourdist': 3897.598127259958, 'tourYear': 2008, 'tourdistOpt': 3897.598127259958}, 3: {'tourLength': 291, 'tourdist': 925631.7134221339, 'tourYear': 2009, 'tourdistOpt': 89218.14117402385}, 4: {'tourLength': 131, 'tourdist': 356680.719377711, 'tourYear': 2011, 'tourdistOpt': 69151.9293533915}, 'name': u'Steve Aoki', 6: {'tourLength': 355, 'tourdist': 914263.7746566971, 'tourYear': 2012, 'tourdistOpt': 100187.90882039357}, 7: {'tourLength': 188, 'tourdist': 546725.6589293461, 'tourYear': 2014, 'tourdistOpt': 78159.49849836492}, 8: {'tourLength': 102, 'tourdist': 278360.29618725023, 'tourYear': 2015, 'tourdistOpt': 65007.757952787164}, 9: {'tourLength': 235, 'tourdist': 769053.0295231593, 'tourYear': 2016, 'tourdistOpt': 86929.62433792242}, 'totalArtistOptTourDist': 515747.87721413205, 'OptArtistPercent': 86.55463586381427, 'totalArtistTourDist': 3835878.8351896

In [34]:
###HERE WE BUILD A LABEL OWNERSHIP CRUSHER FACTORIZED
with open("../../4_miner/labels_aliases_TOTAL.obj", 'rb') as handle:
          tempRelAll= pickle.load( handle)
def find_owning_label(label):
    
    labrels=[]
    if "label-relation-list" in tempRelAll[label]["label"] :

        for relat in tempRelAll[label]["label"]["label-relation-list"]:
            if "direction" in relat and relat["direction"] == "backward":
                print tempRelAll[label]["label"]["name"],"OWNED BY",relat["target"]," ",tempRelAll[relat["target"]]["label"]["name"]
                labrels.append(relat["target"])
                rum_and_whisky=""
                while rum_and_whisky != "SELF": 
                    rum_and_whisky=find_owning_label(relat["target"])
                print "ENDED LOOP"
                labrels.remove(relat["target"])
                print "LABRELS",labrels
                #if labrels[0]==relat["target"]:
                 #   break
                
            else:
                pass
            
        
        if not labrels :
            print "TOP OF THE TREE FOR ",tempRelAll[label]["label"]["name"]
            return "SELF"
        else:
            return labrels
                


artist={}
artist["name"]="Steve Aoki"
artist["mbid"]="a39acc4a-985e-4173-a9ed-f325f2d3bc1c"

with open("../../4_miner/release_mbnz/"+artist["name"]+artist["mbid"]+".obj", 'rb') as handle:
          tempRelz= pickle.load( handle)
#print tempRel["release-list"][0]['label-info-list'][0]['label']['id']
#print tempRel["release-list"][0]['release-event-list'][0]["date"]




for release in tempRelz["release-list"]:
    print"---------------------------"
    print release['label-info-list'][0]['label']['id']
    print release['label-info-list'][0]['label']['name']
    print release['release-event-list'][0]["date"]
    
    #print tempRelAll[release['label-info-list'][0]['label']['id']]["label"]
    try:
        result_lab=find_owning_label(release['label-info-list'][0]['label']['id'])
        print "REZLAB",result_lab 
        print"XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
    except KeyError as e:
        #print e
        pass

EOFError: 

In [43]:
with open("../LiSTGZ.obj", 'rb') as handle:
          LISTGZ= pickle.load( handle)
for ART in LISTGZ:
    print ART


wolves-in-the-throne-room
woods
regal
hot-tears
melt-banana
dan-sartain
shining
scout-niblett
dalek
black-cobra
the-eye-of-time
salsa
ovo
geneva-jacuzzi
expo
lotus-fucker
jeffrey-lewis-los-bolts
wavves
sages-comme-des-sauvages
dj
trophy-wife
mike-watt-the-missingmen
sleepy-sun
chris-cohen
gable
damily
dysrhythmia
deerhoof
id
stuntman
monarch
tacocat
lithics
the-arson-project
musique
sourdure
acid-mother-temple
year-of-no-light
surrender
the-ex
tacos
institute
yonatan-gat
gun-outfit
france
robot-orchestra
frankie-cosmos
sun-araw
baseck
dirty-beaches
kurt
ducktails
unsane
container
cough
kylesa
ar-ker
the-space-lady
reverie
monochrome
salo
bob-log-iii
guerilla-toss
ghold
cobra
red-alert
part-chimp
guantanamo-baywatch
requin-chagrin
shopping
cancelled
bermuda
marisa-anderson
dan-friel
mdou-moctar
low
sete-star-sept
terry-malts
stagger-lee
maserati
marissa-nadler
dead-horses
helen-money
ed-schrader-s-music-beat
purling-hiss
cowtown
health
tune-yards
matchess
weasel-walter
your-old-droog
k

In [24]:
## setup mongo
client = MongoClient()
db = client["bandstour"]
bandsintown =  db["minedArtists"]

# credentials
TOPOGRAM_URL = "https://app.topogram.io"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"

# data
#title = "Foo Fighters bandstour 0.1"
my_nodesdict = {}
my_nodes= []
my_edges = []
my_edgesdict = {}
my_nodesdictGLOBAL={}

#LIVENATION CATALOGUE
liste_salles_LN = json.load(open('./JSON_LIVENATION_VENUE.json'))

liste_salle_parsee_LN={}
for salle in liste_salles_LN:
    if liste_salles_LN[salle]["type"]=="venue":
 
            liste_salle_parsee_LN[slugify(liste_salles_LN[salle]["title"])]={"lat":liste_salles_LN[salle]["position"]["lat"],"lng":liste_salles_LN[salle]["position"]["lng"]}
#print liste_salle_parsee_LN


# date parsing
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
DATETIME_FORMAT_FOR_MSBNZ ='%Y-%m-%d'

#retrieve trip coords for each artist
dict_coords={}
colors_for_nodes_and_edges = get_N_HexCol()

#for artist in db.minedArtists.find({"totalKm": {"$gte": 250000}}) :
for ART in LISTGZ:
 for artist in db.minedArtists.find({"slug" : ART}) :
  title = artist["name"]   
  if os.path.exists(savpath+slugify(artist["name"]+str(artist["mbid"]))+".obj")==True:
        my_nodesdict = {}
        my_nodesdictList=[]
        print "SKIPPED PROCESSING BUT TAKING INTO ACCOUNT ",artist["name"]
        with open(savpath+slugify(artist["name"]+str(artist["mbid"]))+".obj", 'rb') as handle:
          tempL= pickle.load( handle)
    #print 
        tit=tempL[0].encode("utf-8")
        #edgestop=temp[2]
        my_nodesdictList=tempL[1]
        for node in my_nodesdictList:
            my_nodesdict[node["data"]["id"]]=node

        #print my_nodes
        #pass
 else:
    my_nodesdict = {}
    my_nodes= []
    my_edges = []
    my_nodesdictList=[]
    my_edgesdict = {}
    #colors_for_nodes_and_edges = get_N_HexCol()
    
    print artist["name"]
    #print artist
    dates_coords = []
    dates = artist.get("gigs")
    tours = artist.get("tours")    
    singlegigs = artist.get("singleGigs")
    dateini=0
    touring=0
    tour_indice=0
    tour_counter_for_CO2=1
    tour_counter_for_CO2_lock=0
    tmpOLDgig ={}
    with open("../../4_miner/tour_miner/"+slugify(artist["name"]+str(artist["mbid"]))+".obj", 'rb') as handle:
                    tempCO2= pickle.load( handle)
    #print tempCO2
    #print tours
    for tour in tours:
        
        ##WE NEED TO MATCH TOUR INDICES OF CO2 SPECS FILES...
     if tour_counter_for_CO2_lock == 1:
        tour_counter_for_CO2+=1
     tour_counter_for_CO2_lock=0
     tour_color = "#"+colors_for_nodes_and_edges[tour_indice % 20]
     #print tour_color
     #print tour
     gig_indice =0
     dates = tour.get("gigs")
     for date in dates:
        #print date.get("datetime")
        venue = date.get("venue")
        latitude = venue.get("latitude")
        longitude = venue.get("longitude")
        #print venue
        ###concat key
        idd= slugify(venue["name"])+str(int(venue["latitude"]))+str(int(venue["longitude"]))
        
        if idd not in my_nodesdict:
            
            node = {
                    "id" : idd,
                    "name" : slugify(venue["name"]),
                    "lat" : float(venue["latitude"]),
                    "lng" : float(venue["longitude"]),
                    "weight" : float(1),
                    "start" : date.get("datetime").isoformat(),
                    "end" : (date.get("datetime")+timedelta(days=1)).isoformat(),
                    "color" : tour_color,
                    "notes" : "name : "+slugify(venue["name"])+"  \nlat : "+str(float(venue["latitude"]))+"  \nlng :"+str(float(venue["longitude"]))+"  \n weight :"+str(1)+"  \nstart : "+str(date.get("datetime").isoformat())+"  \nend : "+str((date.get("datetime")+timedelta(days=1)).isoformat())+"  \ncolor :"+str(tour_color)+"  \n  <a href=\"https://google.com/search?q="+slugify(venue["name"])+"\" target=\"_blank\">"+slugify(venue["name"])+"</a>"
                #+"  \n[search venue named "+slugify(venue["name"])+" on google](https://google.com/search?q="+slugify(venue["name"])+")"
                    }
        else:
                
                node = {
                    "id" : idd,
                    "name" : slugify(venue["name"]),
                    "lat" : float(venue["latitude"]),
                    "lng" : float(venue["longitude"]),
                    "weight" : math.sqrt(float(int(float(my_nodesdict[idd]["data"].get("weight"))**2)+1)) ,
                    "start" : my_nodesdict[idd]["data"].get("start"),
                    "end" : (date.get("datetime")+timedelta(days=1)).isoformat(),
                    "color" : tour_color,
                    "notes" : "name : "+slugify(venue["name"])+"  \nlat : "+str(float(venue["latitude"]))+"  \nlng :"+str(float(venue["longitude"]))+"  \n weight :"+str((float(my_nodesdict[idd]["data"].get("weight")))**2+1)+"  \nstart : "+str(my_nodesdict[idd]["data"].get("start"))+"  \nend : "+(date.get("datetime")+timedelta(days=1)).isoformat()+"  \ncolor :"+tour_color+"  \n  <a href=\"https://google.com/search?q="+slugify(venue["name"])+"\" target=\"_blank\"> "+slugify(venue["name"])+" </a>"                    #+"  \n[search venue named "+slugify(venue["name"])+" on google](https://google.com/search?q="+slugify(venue["name"])+")"
                    
                    }
        if dateini == 0:
                dateini =1
        else:
                #print my_nodes[-1]["data"]["id"]
                #print dates[gig_indice-1]["distanceToNextGig"]
                
                
                
                notesE=""
                notesE+="distance : "
                try: 
                    notesE+=str(dates[gig_indice]["distanceToNextGig"])
                except KeyError:
                    notesE+="dates[gig_indice][distanceToNextGig] NOT AVAILABLE"
                notesE+=" km  \n"
                notesE+="source : "
                notesE+=str(my_nodes[-1]["data"]["id"])
                notesE+="  target : "
                notesE+=str(node["id"])
                notesE+="  \ndatesource : "
                notesE+=dates[gig_indice -1]["datetime"].isoformat()
                notesE+="  \ndatetarget : "
                notesE+=dates[gig_indice]["datetime"].isoformat()
                notesE+="  \ngroup: "
                notesE+=str(int(tour_indice)+1)
                #notesE+=
                
                
                #print gig_indice
                edge = {
                    "source" : my_nodes[-1]["data"]["id"],
                    "target" : node["id"],
                    "color" :  tour_color,
                    "notes" : notesE,
                    #"weight" : float(e["weight"]),
                    "start" : dates[gig_indice -1]["datetime"].isoformat(),
                    "end" : dates[gig_indice]["datetime"].isoformat(),
                    "group" : str(tour_counter_for_CO2)
                    
                }
                #print edge
                #ADDING EDGES GROUPS/CO2 HERE
                
                if tour["nbGigs"] > 2:
                    try:
                        if tempCO2[tour_counter_for_CO2]['tourdist'] > 1.25*tempCO2[tour_counter_for_CO2]['tourdistOpt']:
                            
                            edge["group"]+=" DASHED2"
                            edge["notes"]+="  \n tournée grandement optimisable  \nDistance parcourue pendant le tour: "+str(tempCO2[tour_counter_for_CO2]['tourdist'])+" km  \n"
                            edge["notes"]+="Distance recalculée: "+str(tempCO2[tour_counter_for_CO2]['tourdistOpt'])+" km  \n"
                            try:
                                edge["notes"]+="Taux d'optimisation "+str((tempCO2[tour_counter_for_CO2]['tourdist']-tempCO2[tour_counter_for_CO2]['tourdistOpt'])/tempCO2[tour_counter_for_CO2]['tourdist'])+" %"
                            except:
                                print "OUPS 2"
                        elif (tempCO2[tour_counter_for_CO2]['tourdist'] < 1.25*tempCO2[tour_counter_for_CO2]['tourdistOpt']) and (tempCO2[tour_counter_for_CO2]['tourdist'] > 1.1*tempCO2[tour_counter_for_CO2]['tourdistOpt']):
                            
                            edge["group"]+=" DASHED1"
                            edge["notes"]+="  \n  tournée optimisable  \nDistance parcourue pendant le tour: "+str(tempCO2[tour_counter_for_CO2]['tourdist'])+" km  \n"
                            edge["notes"]+="Distance recalculée: "+str(tempCO2[tour_counter_for_CO2]['tourdistOpt'])+" km  \n"
                            try:
                                edge["notes"]+="Taux d'optimisation "+str((tempCO2[tour_counter_for_CO2]['tourdist']-tempCO2[tour_counter_for_CO2]['tourdistOpt'])/tempCO2[tour_counter_for_CO2]['tourdist'])+" %"
                            except:
                                print "OUPS 1"
                        elif tempCO2[tour_counter_for_CO2]['tourdistOpt'] > 1.25*tempCO2[tour_counter_for_CO2]['tourdist']:
                            
                            edge["group"]+=" DASHED-2"
                            edge["notes"]+="  \n tournée déjà grandement optimisée  \nDistance parcourue pendant le tour: "+str(tempCO2[tour_counter_for_CO2]['tourdist'])+" km  \n"
                            edge["notes"]+="Distance recalculée: "+str(tempCO2[tour_counter_for_CO2]['tourdistOpt'])+" km  \n"
                            try:
                                edge["notes"]+="Taux d'optimisation "+str((tempCO2[tour_counter_for_CO2]['tourdist']-tempCO2[tour_counter_for_CO2]['tourdistOpt'])/tempCO2[tour_counter_for_CO2]['tourdist'])+" %"
                            except:
                                print "OUPS -2"
                        elif tempCO2[tour_counter_for_CO2]['tourdistOpt'] < 1.25*tempCO2[tour_counter_for_CO2]['tourdist'] and tempCO2[tour_counter_for_CO2]['tourdistOpt'] > 1.1*tempCO2[tour_counter_for_CO2]['tourdist']:
                            
                            edge["group"]+=" DASHED-1"
                            edge["notes"]+="  \n  tournée déjà optimisée  \nDistance parcourue pendant le tour: "+str(tempCO2[tour_counter_for_CO2]['tourdist'])+" km  \n"
                            edge["notes"]+="Distance recalculée: "+str(tempCO2[tour_counter_for_CO2]['tourdistOpt'])+" km  \n"
                            try:
                                edge["notes"]+="Taux d'optimisation "+str((tempCO2[tour_counter_for_CO2]['tourdist']-tempCO2[tour_counter_for_CO2]['tourdistOpt'])/tempCO2[tour_counter_for_CO2]['tourdist'])+" %"
                            except:
                                print "OUPS -1"
                        
                        else:

                            edge["notes"]+="  \nDistance parcourue pendant le tour: "+str(tempCO2[tour_counter_for_CO2]['tourdist'])+" km  \n"
                        tour_counter_for_CO2_lock=1
                    except KeyError as e:
                        print e
                else:
                    #print "TOUR NON COMPTE ", tour_indice, "//",tour_counter_for_CO2
                    continue
                    
                    
                    
                my_edges.append({ "data" : edge })
        my_nodes.append({ "data" : node })   
        my_nodesdict[node["id"]]= { "data" : node }
        gig_indice+=1
    #print my_nodes
    #print "<=============================================>"
    #print my_nodesdict
    #print "<=============================================>"
    #print my_edges
     tour_indice +=1
        #print "coords",latitude,"/",longitude
    for key, value in my_nodesdict.iteritems():
            my_nodesdictList.append(value)
    
    #ADDING LIVENATION DETECTOR

    for node in my_nodesdictList:
        for salle in liste_salle_parsee_LN:
            if fuzz.partial_ratio( node['data']['name'], salle)>=80:
                #print "ONE HEERE"
                if round(node['data']['lat'],2)==round(liste_salle_parsee_LN[salle]["lat"],2) and round(node['data']['lng'],2)==round(liste_salle_parsee_LN[salle]["lng"],2) :
                    #print "CONFIRMED LIVENATION!"
                    node['data']['color']="#ff0000"
                    node['data']["notes"]+="  \nLivenation venue"
        #if liste_salle_parsee_LN
        
        
    #ADDING EDGES WEIGHT
    terp= map(lambda x : str(sorted([x["data"]["source"],x["data"]["target"]])),my_edges)
    
    #print terp
    from collections import Counter
    c= Counter()
    for ter in terp:
        c[ter]+=1
    #print c
    for edge in my_edges:
        if str(sorted([edge["data"]["source"],edge["data"]["target"]])) not in terp:
            print "BUG!!!!"
        else:
            if edge["data"]["source"] == edge["data"]["target"]:
                #print c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))]
                edge["data"]["notes"]+="  \ncounted "+str(c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))])+"  times"
                edge["data"]["weight"]= float(1)
                #print "a"
            else:
                edge["data"]["notes"]+="  \ncounted "+str(c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))])+"  times"
                edge["data"]["weight"]= float(c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))])
                #print "b"
    #print my_edges

    
    
    
    

    titredugraphe =artist["name"]+"/BETA_0.9"+"  \nDistance totale parcourue par l'artiste: "+str(tempCO2["totalArtistTourDist"])+" km"
    titredugraphe += "  \nDistance calculee par Concorde: "+str(tempCO2["totalArtistOptTourDist"])+" km"
    if  "OptArtistPercent" in tempCO2 :
         rateOpt=0
         rateOpt=float(tempCO2["OptArtistPercent"])
         titredugraphe+="  \nPourcentage d'optimisation global calcule:"+str(rateOpt)+" %"
         if rateOpt > 10: 
             titredugraphe+="  \nMARGE D'OPTIMISATION IMPORTANTE"
         elif rateOpt < -10:
              titredugraphe+="  \nTOURNEE DEJA OPTIMISEE"
         else: 
             titredugraphe+="  \nGLOBALEMENT IDENTIQUE"
    
    titredugraphe +="  \nPLAYED AT GZ"     
         
    with open(savpath+slugify(artist["name"]+str(artist["mbid"]))+".obj", 'wb') as handle:
                            pickle.dump([titredugraphe,my_nodesdictList,my_edges], handle, protocol=pickle.HIGHEST_PROTOCOL)    
            
    ###CREATE NODE GLOBAL DICT


!!!
Wolves in the Throne Room
Woods
Regal
Hot Tears
Melt-Banana
Dan Sartain
Shining
Scout Niblett
Dälek
Black Cobra
The eye of time
salsa
OVO
Geneva Jacuzzi
Expo
Lotus Fucker
Jeffrey Lewis & Los Bolts
Wavves
Sages Comme Des Sauvages
Dj
Trophy Wife
mike watt + the missingmen
Sleepy Sun
Chris Cohen
Gablé
Damily
Dysrhythmia
Deerhoof
ID
Stuntman
Monarch
Tacocat
Lithics
The Arson Project
Musique
Sourdure
Acid Mother Temple
Year of No Light
Surrender
The Ex
Tacos
Institute
Yonatan Gat
Gun Outfit
France
Robot Orchestra
Frankie Cosmos
Sun Araw
Baseck
Dirty Beaches
Kurt
Ducktails
Unsane
Container
Cough
Kylesa
Ar Ker
The Space Lady
Reverie
Monochrome
Salo
Bob Log III
Guerilla Toss
Ghold
Cobra
Red Alert
Part Chimp
Guantanamo Baywatch
Requin Chagrin
Shopping
Cancelled
Bermuda
Marisa Anderson
Dan Friel
Mdou Moctar
Low
Sete Star Sept
Terry Malts
Stagger Lee
Maserati
Marissa Nadler
Dead Horses
Helen Money
Ed Schrader's Music Beat
Purling Hiss
Cowtown
Health
tUnE-YaRdS
Matchess
Weasel Walter
Your Old 

In [ ]:
     with open("./my_nodesdictGLOBAL.obj", 'rb') as handle:
          my_nodesdictGLOBAL= pickle.load( handle)

In [6]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

print take(5,my_nodesdictGLOBAL.iteritems())

[('ohio-university39-83', {'data': {'count': 1, 'start': '2016-06-10T18:30:00', 'end': '2016-06-11T18:30:00', 'name': 'ohio-university', 'weight': 1.0, 'lat': 39.3268329, 'lng': -83.0063841, 'notes': '  \n List of the bands that played in this venue:  \nled-zepagain  \n TOTAL: 1 Bands', 'id': 'ohio-university39-83', 'color': '#7f3f3f'}}), ('ohio-university39-82', {'data': {'count': 9, 'start': '2008-04-09T19:00:00', 'end': '2015-11-15T19:00:00', 'name': 'ohio-university', 'weight': 3.3784927944829333, 'lat': 39.324921, 'lng': -82.099709, 'notes': '  \n List of the bands that played in this venue:  \nben-folds  \njosh-ritter  \nmachine-gun-kelly  \nray-wylie-hubbard  \nrayland-baxter  \nthe-blind-boys-of-alabama  \ntyler-farr  \nwiz-khalifa  \nzo  \n TOTAL: 9 Bands', 'id': 'ohio-university39-82', 'color': '#5f7f3f'}}), ('tucson-music-hall32-110', {'data': {'count': 35, 'start': '2007-12-17T00:00:00', 'end': '2017-09-15T19:00:00', 'name': 'tucson-music-hall', 'weight': 6.9258925443875565

In [57]:
# credentials
TOPOGRAM_URL = "http://localhost:3000"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"

In [44]:
# credentials
TOPOGRAM_URL = "https://bandstour.topogram.io"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"

In [5]:
print len(my_nodesdictGLOBAL)

NameError: name 'my_nodesdictGLOBAL' is not defined

In [61]:
# connect to the topogram instance (pass debug=True params for more info )
topogram = TopogramAPIClient(TOPOGRAM_URL) #, debug=True)

# create a new user
try :
    topogram.create_user(USER, PASSWORD)
except ValueError:
    print "> User has already been created."

# login a new user if needed
resp_user_login = topogram.user_login(USER, PASSWORD)
print resp_user_login

assert(resp_user_login["status"] == "success")
assert(resp_user_login["status_code"] == 200)


ConnectionError: HTTPConnectionPool(host='localhost', port=3000): Max retries exceeded with url: /api/%20 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc0b5389310>: Failed to establish a new connection: [Errno 111] Connection refused',))

In [60]:
###IMPORT ALL THE BANDS GRAPHS

nodestop=[]
edgestop=[]
tit=[]
import glob, os

print os.getcwd()

for subdir, dirs, files in os.walk(savpath): 
 for file in files:
  if file.endswith('.obj') == True:
    #print file
    with open(os.path.join(subdir, file), 'rb') as handle:
          temp= pickle.load( handle)
    #print temp 
    tit=temp[0].encode("utf-8")
    edgestop=temp[2]
    nodestop=temp[1]
    if (len(nodestop)>50):

      print len(nodestop),len(edgestop) 
      while True:
        try:
    #        print tit
    
            create_topogram(tit, nodestop, edgestop)
            
        except TypeError:
            continue
        
        
        break
print 'DONE'

ERROR:topogram-client:500 - Error : A topogram with the same name already exists


/home/goonieb/gitrep/bandstour/5_visualizer/topogram-api-clientFOR_TOPO_FROM_BANDSTOUR
123 130
Creating topogram 'Chris Cohen/BETA_0.9  
Distance totale parcourue par l'artiste: 92407.5483559 km  
Distance calculee par Concorde: 74292.8211586 km  
Pourcentage d'optimisation global calcule:19.6030816959 %  
MARGE D'OPTIMISATION IMPORTANTE  
PLAYED AT GZ'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u"Chris Cohen/BETA_0.9  \nDistance totale parcourue par l'artiste: 92407.5483559 km  \nDistance calculee par Concorde: 74292.8211586 km  \nPourcentage d'optimisation global calcule:19.6030816959 %  \nMARGE D'OPTIMISATION IMPORTANTE  \nPLAYED AT GZ", u'userId': u'Bf5svhLniEkkcvts2', u'_id': u'aB67ygv2w4z6MucNF', u'slug': u'chris-cohenbeta_09-distance-totale-parcourue-par-lartiste-924075483559-km-distance-calculee-par-concorde-742928211586-km-pourcentage-doptimisation-global-calcule196030816959-marge-doptimisation-importante-p

ERROR:topogram-client:500 - Error : A topogram with the same name already exists


123 nodes created.
130 edges created.
done. Topogram has been updated. Check it at http://localhost:3000/topograms/aB67ygv2w4z6MucNF
357 442
Creating topogram 'Bob Log III/BETA_0.9  
Distance totale parcourue par l'artiste: 348975.202372 km  
Distance calculee par Concorde: 369088.323832 km  
Pourcentage d'optimisation global calcule:-5.76348156642 %  
GLOBALEMENT IDENTIQUE  
PLAYED AT GZ'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u"Bob Log III/BETA_0.9  \nDistance totale parcourue par l'artiste: 348975.202372 km  \nDistance calculee par Concorde: 369088.323832 km  \nPourcentage d'optimisation global calcule:-5.76348156642 %  \nGLOBALEMENT IDENTIQUE  \nPLAYED AT GZ", u'userId': u'Bf5svhLniEkkcvts2', u'_id': u'AsYMyyCqNsTTzWqsW', u'slug': u'bob-log-iiibeta_09-distance-totale-parcourue-par-lartiste-348975202372-km-distance-calculee-par-concorde-369088323832-km-pourcentage-doptimisation-global-calcule-576348156642-glo

ERROR:topogram-client:500 - Error : A topogram with the same name already exists


...

155 nodes created.
162 edges created.
done. Topogram has been updated. Check it at http://localhost:3000/topograms/TNizzRrqksG2xkYh9
60 78
Creating topogram 'Maserati/BETA_0.9  
Distance totale parcourue par l'artiste: 61467.9250693 km  
Distance calculee par Concorde: 61233.7441138 km  
Pourcentage d'optimisation global calcule:0.380980739547 %  
GLOBALEMENT IDENTIQUE  
PLAYED AT GZ'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u"Maserati/BETA_0.9  \nDistance totale parcourue par l'artiste: 61467.9250693 km  \nDistance calculee par Concorde: 61233.7441138 km  \nPourcentage d'optimisation global calcule:0.380980739547 %  \nGLOBALEMENT IDENTIQUE  \nPLAYED AT GZ", u'userId': u'Bf5svhLniEkkcvts2', u'_id': u'ZvPyziv7CwRoNBxe8', u'slug': u'maseratibeta_09-distance-totale-parcourue-par-lartiste-614679250693-km-distance-calculee-par-concorde-612337441138-km-pourcentage-doptimisation-global-calcule0380980739547-globalement-id

ERROR:topogram-client:500 - Error : A topogram with the same name already exists


60 nodes created.
78 edges created.
done. Topogram has been updated. Check it at http://localhost:3000/topograms/ZvPyziv7CwRoNBxe8
182 285
Creating topogram 'Cancelled/BETA_0.9  
Distance totale parcourue par l'artiste: 940138.242309 km  
Distance calculee par Concorde: 383717.259003 km  
Pourcentage d'optimisation global calcule:59.1850175076 %  
MARGE D'OPTIMISATION IMPORTANTE  
PLAYED AT GZ'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u"Cancelled/BETA_0.9  \nDistance totale parcourue par l'artiste: 940138.242309 km  \nDistance calculee par Concorde: 383717.259003 km  \nPourcentage d'optimisation global calcule:59.1850175076 %  \nMARGE D'OPTIMISATION IMPORTANTE  \nPLAYED AT GZ", u'userId': u'Bf5svhLniEkkcvts2', u'_id': u'BppS9f8B2djKmxffw', u'slug': u'cancelledbeta_09-distance-totale-parcourue-par-lartiste-940138242309-km-distance-calculee-par-concorde-383717259003-km-pourcentage-doptimisation-global-calcule5918501

ERROR:topogram-client:500 - Error : A topogram with the same name already exists


182 nodes created.
285 edges created.
done. Topogram has been updated. Check it at http://localhost:3000/topograms/BppS9f8B2djKmxffw
302 359
Creating topogram 'Sleepy Sun/BETA_0.9  
Distance totale parcourue par l'artiste: 205185.037481 km  
Distance calculee par Concorde: 168812.623853 km  
Pourcentage d'optimisation global calcule:17.7266403412 %  
MARGE D'OPTIMISATION IMPORTANTE  
PLAYED AT GZ'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u"Sleepy Sun/BETA_0.9  \nDistance totale parcourue par l'artiste: 205185.037481 km  \nDistance calculee par Concorde: 168812.623853 km  \nPourcentage d'optimisation global calcule:17.7266403412 %  \nMARGE D'OPTIMISATION IMPORTANTE  \nPLAYED AT GZ", u'userId': u'Bf5svhLniEkkcvts2', u'_id': u'xSQBvwgm34ov677LP', u'slug': u'sleepy-sunbeta_09-distance-totale-parcourue-par-lartiste-205185037481-km-distance-calculee-par-concorde-168812623853-km-pourcentage-doptimisation-global-calcule17

In [39]:
###IMPORT ALL THE BANDS GRAPHS

nodestop=[]
edgestop=[]
tit=[]
import glob, os

print os.getcwd()

for subdir, dirs, files in os.walk(savpath): 
 for file in files:
  if file.endswith('.obj') == True:
    #print file
    with open(os.path.join(subdir, file), 'rb') as handle:
          temp= pickle.load( handle)
    #print temp 
    tit=temp[0].encode("utf-8")
    edgestop=temp[2]
    nodestop=temp[1]
    #print edgestop
    #for edge in edgestop:
    #    assert edge["data
    while True:
        try:
    #        print tit
    
            create_topogram(tit, nodestop, edgestop)
            
        except TypeError:
            continue
        except ValueError:
            pass
        
        break
print 'DONE'
        

/home/goonieb/gitrep/bandstour/5_visualizer/topogram-api-clientFOR_TOPO_FROM_BANDSTOUR
Creating topogram 'Chris Cohen/BETA_0.9  
Distance totale parcourue par l'artiste: 92407.5483559 km  
Distance calculee par Concorde: 74292.8211586 km  
Pourcentage d'optimisation global calcule:19.6030816959 %  
MARGE D'OPTIMISATION IMPORTANTE  
PLAYED AT GZ'


ERROR:topogram-client:500 - Error : A topogram with the same name already exists


> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u"Chris Cohen/BETA_0.9  \nDistance totale parcourue par l'artiste: 92407.5483559 km  \nDistance calculee par Concorde: 74292.8211586 km  \nPourcentage d'optimisation global calcule:19.6030816959 %  \nMARGE D'OPTIMISATION IMPORTANTE  \nPLAYED AT GZ", u'userId': u'BgWEdXXGnvEq2Hodg', u'_id': u'LH8wceHDvSPSFLPYv', u'slug': u'chris-cohenbeta_09-distance-totale-parcourue-par-lartiste-924075483559-km-distance-calculee-par-concorde-742928211586-km-pourcentage-doptimisation-global-calcule196030816959-marge-doptimisation-importante-played-at-gz', u'createdAt': u'2019-09-30T17:59:33.150Z'}, u'statusCode': 200}
topogram ID : LH8wceHDvSPSFLPYv
123 nodes deleted
130 edges deleted
Creating topogram 'Ar Ker/BETA_0.9  
Distance totale parcourue par l'artiste: 0 km  
Distance calculee par Concorde: 0 km  
PLAYED AT GZ'


ERROR:topogram-client:500 - Error : A topogram with the same name already exists


> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u"Ar Ker/BETA_0.9  \nDistance totale parcourue par l'artiste: 0 km  \nDistance calculee par Concorde: 0 km  \nPLAYED AT GZ", u'userId': u'BgWEdXXGnvEq2Hodg', u'_id': u'skNcYcTJBpFNG45sL', u'slug': u'ar-kerbeta_09-distance-totale-parcourue-par-lartiste-0-km-distance-calculee-par-concorde-0-km-played-at-gz', u'createdAt': u'2019-09-30T18:00:09.448Z'}, u'statusCode': 200}
topogram ID : skNcYcTJBpFNG45sL
1 nodes deleted
1 nodes created.
Creating topogram 'François Virot/BETA_0.9  
Distance totale parcourue par l'artiste: 543.878136546 km  
Distance calculee par Concorde: 543.878136546 km  
Pourcentage d'optimisation global calcule:0.0 %  
GLOBALEMENT IDENTIQUE  
PLAYED AT GZ'


...

ERROR:topogram-client:500 - Error : A topogram with the same name already exists


> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u"SNEAKS/BETA_0.9  \nDistance totale parcourue par l'artiste: 9893.91610639 km  \nDistance calculee par Concorde: 7535.02061277 km  \nPourcentage d'optimisation global calcule:23.8418788704 %  \nMARGE D'OPTIMISATION IMPORTANTE  \nPLAYED AT GZ", u'userId': u'BgWEdXXGnvEq2Hodg', u'_id': u'Zv82LgSBZjtCwJSHE', u'slug': u'sneaksbeta_09-distance-totale-parcourue-par-lartiste-989391610639-km-distance-calculee-par-concorde-753502061277-km-pourcentage-doptimisation-global-calcule238418788704-marge-doptimisation-importante-played-at-gz', u'createdAt': u'2019-09-30T18:02:52.507Z'}, u'statusCode': 200}
topogram ID : Zv82LgSBZjtCwJSHE
21 nodes deleted
20 edges deleted
21 nodes created.
20 edges created.
done. Topogram has been updated. Check it at https://bandstour.topogram.io/topograms/Zv82LgSBZjtCwJSHE
Creating topogram 'Zu/BETA_0.9  
Distance totale parcourue par l'artiste: 31387.836

ERROR:topogram-client:500 - Error : A topogram with the same name already exists


> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u"Zu/BETA_0.9  \nDistance totale parcourue par l'artiste: 31387.8361394 km  \nDistance calculee par Concorde: 27474.8247027 km  \nPourcentage d'optimisation global calcule:12.4666492438 %  \nMARGE D'OPTIMISATION IMPORTANTE  \nPLAYED AT GZ", u'userId': u'BgWEdXXGnvEq2Hodg', u'_id': u'by73XN8AsePHEGqkB', u'slug': u'zubeta_09-distance-totale-parcourue-par-lartiste-313878361394-km-distance-calculee-par-concorde-274748247027-km-pourcentage-doptimisation-global-calcule124666492438-marge-doptimisation-importante-played-at-gz', u'createdAt': u'2019-09-30T18:03:05.231Z'}, u'statusCode': 200}
topogram ID : by73XN8AsePHEGqkB
89 nodes deleted
95 edges deleted


ConnectionError: ('Connection aborted.', BadStatusLine('No status line received - the server has closed the connection',))

In [50]:
###IMPORT THE GLOBAL VENUE  GRAPH
my_nodesdictList=[]
nodestopo=[]
edgestopo=[]
with open("./my_nodesdictGLOBAL.obj", 'rb') as handle:
          my_nodesdictGLOBAL= pickle.load( handle)
        
for key, value in my_nodesdictGLOBAL.iteritems():
            my_nodesdictGLOBAL[key]["data"].pop('count')
        
for key, value in my_nodesdictGLOBAL.iteritems():
            my_nodesdictList.append(value)
        
nodestopo=sorted(my_nodesdictList, key= lambda (v) : v['data']['weight'], reverse=True)[:5000]
print len(nodestopo)



title="global venue graph /BETA 0.3"

while True:
        try:
            print nodestopo
            create_topogram(title, nodestopo, edgestopo)
        
        except TypeError:
            continue
        break
        

ERROR:topogram-client:500 - Error : A topogram with the same name already exists
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


5000 nodes deleted
5000 nodes created.


ValueError: No JSON object could be decoded

In [14]:
with open("./bands_dicts/the-real-mckenzies01612542-01ec-4fc4-a9ad-b6a6e33f11d7.obj", 'rb') as handle:
          DEBUG= pickle.load( handle)


In [18]:
print DEBUG[:2]


[u"The Real McKenzies/BETA_0.8  \nDistance totale parcourue par l'artiste: 180691.650535 km  \nDistance calculee par Concorde: 156448.795249 km  \nPourcentage d'optimisation global calculee:13.4166992303 %  \nMARGE D'OPTIMISATION IMPORTANTE", [{'data': {'end': '2015-03-27T20:30:00', 'name': 'the-horseshoe-tavern', 'weight': 1.0, 'color': '#7f663f', 'notes': 'name : the-horseshoe-tavern  \nlat : 43.6490438  \nlng :-79.395913  \n weight :1  \nstart : 2015-03-26T20:30:00  \nend : 2015-03-27T20:30:00  \ncolor :#7f663f  \n  <a href="https://google.com/search?q=the-horseshoe-tavern" target="_blank">the-horseshoe-tavern</a>', 'start': '2015-03-26T20:30:00', 'lat': 43.6490438, 'lng': -79.395913, 'id': 'the-horseshoe-tavern43-79'}}, {'data': {'end': '2012-05-09T20:00:00', 'name': 'the-pyramid-cabaret', 'weight': 1.0, 'color': '#723f7f', 'notes': 'name : the-pyramid-cabaret  \nlat : 49.890735  \nlng :-97.137711  \n weight :1  \nstart : 2012-05-08T20:00:00  \nend : 2012-05-09T20:00:00  \ncolor :#